## Segment a sparse 3D image with a single material component  

The goal of this notebook is to develop a 3D segmentation algorithm that improves segmentation where features are detected.

**Data:** AM parts from Xuan Zhang. 

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import sys
import time
import seaborn as sns
import pandas as pd

import cupy as cp
from tomo_encoders import Patches
from tomo_encoders.misc import viewer
from tomo_encoders import DataFile
from tomo_encoders.reconstruction.recon import fbp_filter
# from tomo_encoders.misc.voxel_processing import cylindrical_mask, normalize_volume_gpu
# from cupy.fft import rfft, irfft, rfftfreq
from cupyx.scipy.fft import rfft, irfft, rfftfreq, get_fft_plan

In [2]:
nz = 32
n = 2176
ntheta = 1500
# arguments to recon_chunk2: data, theta, center, p3d
data = cp.random.normal(0,1,(ntheta, nz, n)).astype(np.float32)

In [3]:
for i in range(10):
    data = fbp_filter(data, TIMEIT = True)

n: 2176, n_pad: 3264
pad_left: 544, pad_right: 544


/home/atekawade/anaconda3/envs/tf2GPU/lib/python3.9/site-packages/cupyx/scipy/fftpack/_fft.py:152: UserWarning: cuFFT plan cache is disabled on CUDA 11.1 due to a known bug, so performance may be degraded. The bug is fixed on CUDA 11.2+.
  cache = get_plan_cache()


TIME fbp_filter: 212.07 ms
n: 2176, n_pad: 3264
pad_left: 544, pad_right: 544
TIME fbp_filter: 45.37 ms
n: 2176, n_pad: 3264
pad_left: 544, pad_right: 544
TIME fbp_filter: 41.19 ms
n: 2176, n_pad: 3264
pad_left: 544, pad_right: 544
TIME fbp_filter: 40.64 ms
n: 2176, n_pad: 3264
pad_left: 544, pad_right: 544
TIME fbp_filter: 40.59 ms
n: 2176, n_pad: 3264
pad_left: 544, pad_right: 544
TIME fbp_filter: 40.73 ms
n: 2176, n_pad: 3264
pad_left: 544, pad_right: 544
TIME fbp_filter: 43.51 ms
n: 2176, n_pad: 3264
pad_left: 544, pad_right: 544
TIME fbp_filter: 49.73 ms
n: 2176, n_pad: 3264
pad_left: 544, pad_right: 544
TIME fbp_filter: 41.57 ms
n: 2176, n_pad: 3264
pad_left: 544, pad_right: 544
TIME fbp_filter: 41.59 ms


In [4]:
data.shape

(1500, 32, 2176)

In [6]:
[ntheta, nz, n] = data.shape
n_pad = n*(1 + 0.25*2) # 1/4 padding
n_pad = int(np.ceil(n_pad/8.0)*8.0) 
pad_left = int((n_pad - n)//2)
pad_right = n_pad - n - pad_left
print(f'n: {n}, n_pad: {n_pad}')
print(f'pad_left: {pad_left}, pad_right: {pad_right}')

n: 2176, n_pad: 3264
pad_left: 544, pad_right: 544


In [7]:
start_gpu = cp.cuda.Event(); end_gpu = cp.cuda.Event(); start_gpu.record()
stream = cp.cuda.Stream()
with stream:
    data = cp.pad(data, ((0,0),(0,0),(pad_left,pad_right)), mode = 'edge')
end_gpu.record(); end_gpu.synchronize(); t_meas = cp.cuda.get_elapsed_time(start_gpu,end_gpu)
print(f"overhead for making padded array: {t_meas:.2f} ms")        

overhead for making padded array: 4.57 ms


In [8]:
start_gpu = cp.cuda.Event(); end_gpu = cp.cuda.Event(); start_gpu.record()
stream = cp.cuda.Stream()
with stream:
    plan_fwd = get_fft_plan(data, axes=2, value_type='R2C')  # for batched, C2C, 2D transform
    plan_inv = get_fft_plan(rfft(data,axis=2), axes=2, value_type='C2R')  # for batched, C2C, 2D transform
end_gpu.record(); end_gpu.synchronize(); t_meas = cp.cuda.get_elapsed_time(start_gpu,end_gpu)
print(f"overhead for making fft plan?: {t_meas:.2f} ms")        

overhead for making fft plan?: 16.96 ms


In [9]:
start_gpu = cp.cuda.Event(); end_gpu = cp.cuda.Event(); start_gpu.record()
with plan_fwd:
    
    # filter mask
    t = rfftfreq(data.shape[2])
    wfilter = t.astype(cp.float32) #* (1 - t * 2)**3  # parzen

    data0 = wfilter*rfft(data, axis=2)

with plan_inv:
    data[:] = irfft(data0, axis=2)

    #     for k in range(data.shape[0]):
    #         data[k] = irfft(wfilter*rfft(data[k], axis=1), axis=1)

end_gpu.record(); end_gpu.synchronize(); t_meas = cp.cuda.get_elapsed_time(start_gpu,end_gpu)
print(f"time for applying filter: {t_meas:.2f} ms")        

time for applying filter: 29.41 ms
